In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2023-10-13


Found rundate 2023-10-12 10:00:02 < 2023-10-13 at commit 742a263a; returning commit b75a18c


'2023-10-13 10:00:02'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2023-10-13 10:00:02, NJSP was reporting 451 YTD deaths
Current YTD Deaths (2024-10-13 10:00:01-04:00): 507
Previous year YTD Deaths (adjusted; 2023-10-13 10:00:02): 451.34379992179043
Projected 2024 total: 676.5782139812454
12.3% change


FAUQStats cache miss: 2023, Fri Oct 13 10:00:02 EDT 2023


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,HIGHWAY,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,INJURIES,STREET,dt
ACCID,,,,,,,,,,,,,,
12249,07,Essex,0714,Newark City,NaN,Mulberry St,1.0,0.0,0.0,1.0,0.0,NaN,Mulberry St,2023-01-01 01:49:00-05:00
12251,01,Atlantic,0112,Hamilton Twsp,322,State Highway 322 MP 46.4,1.0,1.0,0.0,0.0,0.0,0.0,NaN,2023-01-02 09:26:00-05:00
12253,12,Middlesex,1217,Piscataway Twsp,NaN,South Randolphville Rd,1.0,1.0,0.0,0.0,0.0,0.0,South Randolphville Rd,2023-01-02 19:41:00-05:00
12254,02,Bergen,0221,Garfield City,NaN,Outwater Ln,1.0,0.0,0.0,1.0,0.0,0.0,Outwater Ln,2023-01-03 20:52:00-05:00
12267,07,Essex,0714,Newark City,NaN,Washington St,1.0,0.0,0.0,1.0,0.0,NaN,Washington St,2023-01-04 06:21:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12738,04,Camden,0412,Cherry Hill Twsp,70,State Highway 70,1.0,0.0,0.0,0.0,1.0,NaN,NaN,2023-10-09 21:17:00-04:00
12733,07,Essex,0709,Irvington Town,444,Garden State Parkway MP 143,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2023-10-09 21:24:00-04:00
12736,17,Salem,1710,Pittsgrove Twsp,553,County 553 MP 28.8,1.0,0.0,1.0,0.0,0.0,NaN,NaN,2023-10-11 05:32:00-04:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,24,2,14,3,42
Bergen,14,4,13,2,31
Burlington,25,4,4,1,34
Camden,14,3,13,0,30
Cape May,5,0,0,0,5
Cumberland,16,6,3,0,20
Essex,13,3,18,0,34
Gloucester,12,3,2,0,16
Hudson,7,2,3,0,11


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

78.2% through the year, 21.8% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

,,prv_ytd,prv_end,cur_ytd
county,type,,,
Atlantic,driver,15,18,24
Bergen,driver,17,20,14
Burlington,driver,16,26,25
Camden,driver,15,18,14
Cape May,driver,4,4,5
...,...,...,...,...
Salem,crashes,9,11,12
Somerset,crashes,15,21,13
Sussex,crashes,4,6,11


In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,55,6,28,2,20
Bergen,41,2,17,8,20
Burlington,53,1,39,6,7
Camden,44,1,17,4,23
Cape May,7,0,5,0,0
Cumberland,23,0,17,7,4
Essex,52,0,17,5,31
Gloucester,20,0,14,4,3
Hudson,14,0,9,6,4


In [9]:
projected.to_csv(PROJECTED_CSV)